# Train Ambient PM25 GNN

In [2]:
import os
from pathlib import Path


project_dp = Path('/content/drive/MyDrive/smoke-signals')
os.chdir(project_dp)
print(f"Current working directory: {Path.cwd()}")


Current working directory: /content/drive/MyDrive/smoke-signals


## Network Repository Installaion


In [3]:
!dpkg -i cuda-keyring_1.1-1_all.deb
!apt update
!apt install -y nvidia-open cuda-toolkit-12-8


(Reading database ... 126332 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) over (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pa

In [4]:
!nvcc --version
!nvidia-smi


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0
Mon Apr 21 05:59:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0       

## Install Dependencies

In [5]:
!pip install torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

### Other Dependencies

In [6]:
!pip install arrow bresenham torch_geometric metpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.6 MB/s eta 0:00:00


In [7]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html


Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 135.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.6 MB/s eta 0:00:00


In [8]:
import pm25_forecasting_gnn


%run -m pm25_forecasting_gnn.train_ambient


cuda


AssertionError: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:2940: UserWarning: Unknown failure executing module: <pm25_forecasting_gnn.train>
  warn('Unknown failure executing module: <%s>' % mod_name)


In [7]:
import pickle
import sys
from pathlib import Path

# Constants
PM25GNN_DIR = Path().cwd().parent
PROJ_DIR = PM25GNN_DIR.parent
sys.path.append(str(PM25GNN_DIR))
RAW_DATA_DIR = PM25GNN_DIR / "data" / "raw"
CITY_FP = RAW_DATA_DIR / "locations.txt"
ALTITUDE_FP = RAW_DATA_DIR / "alt.pkl"

with open(ALTITUDE_FP, "rb") as f:
    my_dict = pickle.load(f)

# Inspect types of keys and values
key_types = set(type(key) for key in my_dict.keys())
value_types = set(type(value) for value in my_dict.values())

print("Key types:", key_types)  # Key types: {<class 'str'>}
print("Value types:", value_types)  # Value types: {<class 'int'>}

Key types: {<class 'tuple'>}
Value types: {<class 'numpy.float64'>}


In [8]:
# Get the first key
sample_key = next(iter(my_dict.keys()))

# Check it's a tuple
print(type(sample_key))  # <class 'tuple'>
print(isinstance(sample_key, tuple))  # True

<class 'tuple'>
True


In [9]:
# Examine elements inside the tuple key
sample_key = next(iter(my_dict.keys()))
element_types = [type(element) for element in sample_key]
print(element_types)  # [<class 'int'>, <class 'str'>]

# More readable version
for i, element in enumerate(sample_key):
    print(f"Element {i} type: {type(element)}")

[<class 'numpy.float64'>, <class 'numpy.float64'>]
Element 0 type: <class 'numpy.float64'>
Element 1 type: <class 'numpy.float64'>
